<a href="https://colab.research.google.com/github/abergues/SalsaApp/blob/eero/create_gsheet_for_labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Google Spreadsheets from the DanceApp output_op/clip_name folder 
This spreadsheet should 
* have the name of the clip_name
* Contain all frame numbers mathcing jpg images. 
* Column names for (leader, follower, hands, feet, turn,) figure


##Todo#
* make a check for existing file. they should not be overwritten.
* add some simple layout
* move files automatically to the right folder. Now they end up in google drive root.


https://github.com/googleapis/google-api-python-client

Unfortunately, I just found out that the Google Sheets API, which I had planned to use for creating the spreadsheet where we can add all image frames we want to
label, does not support adding images. There are two ways around this:
1. Add an URL to the image
1. Use a JavaScript Macro to read the folder and add images.


## Authorize user

In [ ]:
#### AUTHENTICATE USER/APP on Google

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
import gspread
from oauth2client.client import GoogleCredentials
import os
from os.path import exists, join, basename, splitext
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
import shutil                     # used for copying files in the os.

gc = gspread.authorize(GoogleCredentials.get_application_default())

drive.mount('/gdrive')   
# Google Drive root directory
root_path = "/gdrive/My Drive/DSR/DanceApp/output_op"

os.chdir(root_path)
print("\n", "Working directory is now ", os.getcwd())
# print("All files (videos etc) should be placed in ", root_path, "\n")




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).

 Working directory is now  /gdrive/My Drive/DSR/DanceApp/output_op


## Define which video to label.
Just copy the string from 
[preprocess_videos_to_openpose](https://colab.research.google.com/drive/1pCOJ-Pk2V0Mr_X9Th9xLSRjmwX6AFfjk#scrollTo=2otdDKMgrTAb&line=1&uniqifier=1
)

In [ ]:
YOUTUBE_ID, clip_name, start_time, end_time = ["Xm8w6L1bFoQ", "Suzie_Q_ToeTap_HookStep_GhettoFab" , "00:10" , "01:30"]

print("Youtube_id is ", YOUTUBE_ID, " and clip_name is ", clip_name, ".")
print("These are used to put the googlesheet in right folder.")

Youtube_id is  Xm8w6L1bFoQ  and clip_name is  Suzie_Q_ToeTap_HookStep_GhettoFab .
These are used to put the googlesheet in right folder.


In [ ]:
 # print(os.listdir())

os.chdir(root_path + "/" + clip_name)
print("\n", "Working directory is now ", os.getcwd())
paths_jpg = glob.glob("f*.jpeg")
paths_jpg = sorted(paths_jpg, reverse=False)
number_of_frames = len(paths_jpg)
print("there are ", number_of_frames, " jpg files to label")
print("starting with: ", paths_jpg[0], " and ending with ", paths_jpg[-1]  )

# these are used in the cells of the sheet
frames_list = [] 
for frame in paths_jpg:
  frame_nr, _ = split(r'[\.]' , frame)
  frames_list.append(frame_nr)

 
date1 = date.today().strftime("%Y.%m.%d")
print("today is =", date1)

# # Give the new sheet a name
sheet_name = "labels_for_" + YOUTUBE_ID    


 Working directory is now  /gdrive/My Drive/DSR/DanceApp/output_op/Suzie_Q_ToeTap_HookStep_GhettoFab
there are  452  jpg files to label
starting with:  frame-000005.jpeg  and ending with  frame-002260.jpeg


## Create the google spreadsheet


In [ ]:
#### CREATE THE SPREADSHEET
# Check : https://python-everything.com/post/87/Google-Sheets-%26-Python:-How-to-read%2C-change-and-add-data%21.html
import os
from datetime import date
from sys import exit
import time

def create_spreadsheet():
   # TODO: this condition is not working yet. Need to use google URL.
    if not os.path.isfile(sheet_name):
      sh = gc.create(sheet_name)  
    else:  
      print("The sheet exists already. Do you want to replace it?")
      choice = input("press Y or n")
      if choice == "Y": 
        sh = gc.create(sheet_name)
      else:
        sys.exit(0)
 
    # Open our new sheet and add some data.
    ws = gc.open(sheet_name).sheet1
    #          (row, col, text)
    ws.update_cell(1,1 , "Labels for Video")
    ws.update_cell(2,1 ,"Youtube_id is" )
    ws.update_cell(2,3 ,YOUTUBE_ID )
    ws.update_cell(3,1 ,"The Video is in folder" )
    ws.update_cell(3,3 ,clip_name )
    ws.update_cell(4,1 ,"file created on" )
    ws.update_cell(4,3 ,date1 )

    ws.update_cell(6,1 ,"Framenumber")
    ws.update_cell(6,2 ,"Timecode")
    ws.update_cell(6,3 ,"Figure1")
    ws.update_cell(6,4 ,"Figure2")
    ws.update_cell(6,5 ,"Variation1")
    ws.update_cell(6,6 ,"Variation2")
    ws.update_cell(6,7 ,"URL")
    ws.update_cell(6,8 ,"Problem")
    ws.update_cell(6,9 ,"TODO")
    
    # worksheet.update_cells("A1:I9")
      
    # get number of frames.
    # use os.   get names of all jpg in folder

    # calculate the timecode (not sure we need this)

    # find and add image.  (This is Impossible with colab and python) 
    # It is only possible to write an url to a cell
  
    # Create columns
    # import random
    for i, name in enumerate(frames_list):
        on_row = i + 7
        time.sleep(0.2)
        # Google Sheets api has a quota that must not be exceeded.
        # possible solution: create a dataframe, dump this to cell content.
        # update all at once.
        ws.update_cell(on_row, 1 , frames_list[i])

    # worksheet.update_cells(cell_list)
    # Go to https://sheets.google.com to see your new spreadsheet.
create_spreadsheet()